# Script 1: Ensemble 
### This script shows how to caluclate and plot ensemble means. 

## Content:
### Section 1 includes the packages, PMIP model lists and some basic codes used to find the right directory and filenames. 
### Section2 provides the functions and options for calculating ensemble means
### Section 3 provides the options for potting 
### Section 4 shows some examples

# Section 1: Importing packages and defining general information
This section has three parts. The first one lists and imports the packages that are required in calculating and plotting. The second one provides the full lists of models in PMIP4 and PMIP3. The third one defines a set of functions to find the filenames of the files that have the variable in the experiment.

### No change should be maken in this section, unless you have some necessary modifications. 


In [1]:
'''Import packages for loading data, analysing, and plotting'''
# This cell imports the necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from netCDF4 import Dataset
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import os
import matplotlib.colors as colors
import scipy
from sklearn.metrics import mean_squared_error

/Users/annie/anaconda3/envs/an/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
'''Define pimp generations and model names'''
# This cell lists the models involved in PMIP4 and PMIP3
pmip = {}
pmip['PMIP4'] = ['ACCESS-ESM1-5',
                 'AWI-ESM-1-1-LR',
                 'AWI-ESM-2-1-LR',
                 'CESM2',
                 'CNRM-CM6-1',
                 'EC-Earth3-LR',
                 'FGOALS-f3-L',
                 'FGOALS-g3',
                 'GISS-E2-1-G',
                 'HadGEM3-GC31-LL',
                 'INM-CM4-8',
                 'IPSL-CM6A-LR',
                 'MIROC-ES2L',
                 'MPI-ESM1-2-LR',
                 'MRI-ESM2-0',
                 'NESM3',
                 'NorESM1-F',
                 'NorESM2-LM',
                 'UofT-CCSM-4']
pmip['PMIP3'] = ['BCC-CSM1-1',
                 'CCSM4',
                 'CNRM-CM5',
                 'CSIRO-Mk3L-1-2',
                 'CSIRO-Mk3-6-0',
                 'EC-EARTH-2-2',
                 'FGOALS-g2',
                 'FGOALS-s2',
                 'GISS-E2-R',
                 'HadGEM2-CC',
                 'HadGEM2-ES',
                 'IPSL-CM5A-LR',
                 'MIROC-ESM',
                 'MPI-ESM-P',
                 'MRI-CGCM3']
               

In [ ]:
'''Define calculating functions'''
# This cell sets up the routine to find and list filenames that the files have the variable(s)

#This function returns all available experiment names
def experimentlist():
    exps = []
    file_path = "DATA" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function returns all available model names in the target experiment 
def modellist(experiment_name):
    models = []
    file_path = "DATA/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname = f.split("_")[0]
            models.append(mname)
    return models

#This function returns all available filenames in the target experiment 
def filenamelist(experiment_name):
    filenames = []
    file_path = "DATA/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff = 'DATA/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function retruns the models that have the variable in the target experiment
def identify_ensemble_members(variable_name,experiment_name):
    datadir = "DATA/%s" %(experiment_name)
    ensemble_members = !scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function lists excat model name
def extract_model_name(filename):
    file_no_path = filename.rpartition("/")
    file_strings = file_no_path[2].partition("_")
    model_name = file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems = identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict = {extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)] = mem
    return ens_mems_dict
    



# Section 2: Calculating functions
This section defines two functions that can return ensemble (mean) and ensemble (mean) change by choosing appripriate input variables, respectively. Results are regridded to 1x1 deg resolution (i.e. 180x360 grids). Detailed explaination and guidelines are given below.
### If you don't want to create a netCDF file to save the outputs, remove the lines noted by "#remove this line if you don't want to save the resluts" in functions

Function have five inputs:

pmip_v: Define the PMIP generation as "PMIP4" or "PMIP3" 

experiment_name: Define the name of the target experiment, e.g. "midHolocene". If you want to use calendar adjusted variables, you need to add "-cal-adj" after the experimental name, e.g. "midHolocene-cal-adj".

variable_name: Define the name of the target variable, e.g. "tas_spatialmean_ann"

savedirct: Define the directory to save output files. #remove this input if you don't want to save the results into a netCDF file.

choice: Define the returned variable choices by "model", "mean" and "both". 
"model" will save and return the model list and a directory including all absolute values of (or changes in) the rarget variable in the target experimental simulations from each individual model; Output model_list, DATA.

"mean" will save and return the ensemble mean and ensemble standard deviation of the absolute values of (or the changes in) the rarget variable; Output model_list, ave, std

"both" will save and return the outputs from both "model" and "mean"; Output model_list, ave, std, DATA

Outputs:
model_list: Full list of models included in the ensemble
ave: Ensemble mean of the variable (change)
std: Ensemble standard deviation of the variable (change)
DATA: A dtrectoy including the variable (change) simulated by each model, named same as the model name.

## Function 1: calculation_ensemble
calculation_ensemble_change calculates and returns regrided ensemble mean (or individual models), absolute values of the rarget variable in the target experiment. Outputs will be saved in as a netCDF file named 'PMIP_generation'_'experiment_name'_'variable_name'_emsemble_mean.nc" in your chosen directory. 



In [ ]:
def creaefile(pmip_v,experiment_name,variable_name):
    savefile = '%s_%s_%s_emsemble_mean.nc' %(pmip_v,experiment_name,variable_name) 
    lonS = np.arange(-179.5, 180., 1.0) 
    d = Dataset(filename,'w',format='NETCDF4') 
    d.createDimension('lon',360)  
    d.createDimension('lat',180)  
    d.createVariable("lon",'f',("lon"))  
    d.createVariable("lat",'f',("lat"))  
    d.variables['lat'][:] = latS     
    d.variables['lon'][:] = lonS   
    d.close()
    return savefile

In [ ]:
def calculation_ensemble(pmip_v,experiment_name,variable_name,choice): 
    modelfile = '%s_%s_%s_emsemble_mean_models.csv' %(pmip_v,experiment_name,variable_name)
    # Create a netCDF file to save the outputs
    savefile = creaefile(pmip_v,experiment_name,variable_name) #remove this line if you don't want to save the resluts
    d = Dataset(svefile,'a') #remove this line if you don't want to save the resluts
    model_list = []
    DATA = {}
    var_dict = ensemble_members_dict(variable_name,experiment_name)
    grid_1x1 = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90., 1.0)),
                         'lon': (['lon'], np.arange(-179.5, 180, 1.0))})
    for gcm in A_dict:
            if gcm in pmip[pmip_v]:
                model_list.append(gcm)              
                expt_file = xr.open_dataset(var_dict.get(gcm),decode_times=False)
                var = expt_file[variable_name]
                this_regridder = xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
                var_1x1 = this_regridder(var)               
                DATA[gcm] = var_1x1
                if choice == 'model': #remove this line if you don't want to save the resluts
                    d.createVariable(gcm,'f',("lat",'lon')) #remove this line if you don't want to save the resluts
                    d.variables[gcm][:] = var_1x1   #remove this line if you don't want to save the resluts                  
    pd.DataFrame(model_list).to_csv(modelfile)
    if choice == 'mean' or 'both':
        data = np.array(DATA)
        ave = np.average(data,axis=0)
        std = np.std(data,axis=0)
        d.createVariable('ave','f',("lat",'lon'))  #remove this line if you don't want to save the resluts
        d.variables['ave'][:] = ave  #remove this line if you don't want to save the resluts
        d.createVariable('std','f',("lat",'lon'))  #remove this line if you don't want to save the resluts
        d.variables['std'][:] = std #remove this line if you don't want to save the resluts
    d.close()    #remove this line if you don't want to save the resluts
    if choice == 'mean':
        return model_list,ave,std
    if choice == 'model':
        return model_list,DATA
    if choice == 'both':
        return model_list,ave,std,DATA



## Function 2: calculation_ensemble_change
calculation_ensemble_change calculates and returns regrided ensemble mean of absolute changes in the target variable in the target experiment. Outputs will be saved in as a netCDF file named "<PMIP_generation>_<experiment_name>_<variable_name>_emsemble_mean.nc" in your chosen directory. 




In [2]:
def calculation_ensemble_change(pmip_v,experiment_name,variable_name,savedirct,choice):
    modelfile = '%s_%s_%s_emsemble_mean_change_models.csv' %(savedirct,pmip_v,experiment_name,variable_name)
    # Create a netCDF file to save the outputs
    savefile = creaefile(pmip_v,experiment_name,variable_name) #remove this line if you don't want to save the resluts
    d = Dataset(svefile,'a') #remove this line if you don't want to save the resluts
    model_list = []
    DATA = {}
    A_dict = ensemble_members_dict(variable_name,experiment_name)
    B_dict = ensemble_members_dict(variable_name,'piControl')
    grid_1x1 = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90., 1.0)),
                         'lon': (['lon'], np.arange(-179.5, 180, 1.0))})
    for gcm in A_dict:
        if gcm in B_dict:
                if gcm in pmip[pmip_v]:
                    model_list.append(gcm)              
                    expt_a_file = xr.open_dataset(A_dict.get(gcm),decode_times=False)
                    expt_a = expt_a_file[variable_name]
                    expt_b_file = xr.open_dataset(B_dict.get(gcm),decode_times=False)
                    expt_b = expt_b_file[variable_name]
                    diff = expt_a - expt_b
                    this_regridder = xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
                    diff_1x1 = this_regridder(diff)               
                    DATA[gcm] = diff_1x1
                    if choice == 'model':
                        d.createVariable(gcm,'f',("lat",'lon')) #remove this line if you don't want to save the resluts
                        d.variables[gcm][:] = diff_1x1   #remove this line if you don't want to save the resluts                  
    pd.DataFrame(model_list).to_csv(modelfile)
    if choice == 'mean' or 'both':
        data = np.array(DATA)
        ave = np.average(data,axis=0)
        std = np.std(data,axis=0)
        d.createVariable('ave','f',("lat",'lon'))  #remove this line if you don't want to save the resluts
        d.variables['ave'][:] = ave  #remove this line if you don't want to save the resluts
        d.createVariable('std','f',("lat",'lon'))  #remove this line if you don't want to save the resluts
        d.variables['std'][:] = std  #remove this line if you don't want to save the resluts
    d.close()    
    if choice == 'mean':
        return model_list,ave,std
    if choice == 'model':
        return model_list,DATA
    if choice == 'both':
        return model_list,ave,std,DATA




## Calculating the ensemble (mean)

Choose the appropriate inputs. Available inputs are listed afterwards

In [9]:
pmip_v = "PMIP4" #"PMIP4" or "PMIP3"

In [4]:
experiment_name = "midHolocene" 
#       Period              Experiment name            Name of experiment with calendar adjustment applied
#     mid-Holocene           midHolocene                  midHolocene-cal-adj
#     Last Interglacial       lig127k                     lig127k-cal-adj

In [5]:
variable_name = "tas_spatialmean_ann" #e.g. the spatial mean of annual mean surface air temperature (deg C)
# Full list of available variable names can be found in variablenamelist.csv


# The spatial mean variables are named as <variable>_spatialmean_<season>
# commonly used variables:
# tas: surface air temperature (deg C)
# pr: precipitation rate (mm/day)
# psl: sea level pressure (hPa)

# Examples:
# tas_spatialmean_ann: the spatial mean of annual mean surface air temperature (deg C)
# tas_spatialmean_djf: the spatial mean of Dec-Jan-Feb mean surface air temperature (deg C)
# tas_spatialmean_jja: the spatial mean of Jun-Jul-Aug mean surface air temperature (deg C)
# pr_spatialmean_ann: the spatial mean of annual mean precipitation rate (mm/day)
# pr_spatialmean_djf: the spatial mean of Dec-Jan-Feb precipitation rate (mm/day)
# pr_spatialmean_jja: the spatial mean of Jun-Jul-Aug precipitation rate (mm/day)


In [7]:
choice = "mean" # "mean", "model" or "both"

# model: save and return the model list and a directory including all absolute values of (or changes in) the rarget variable in the target experimental simulations from each individual model;
# model_list,aDATA = calculation_ensemble_change(pmip_v,experiment_name,variable_name,savedirct,"model")


# mean: save and return the ensemble mean and ensemble standard deviation of the absolute values of (or the changes in) the rarget variable;
# model_list,ave,std = calculation_ensemble_change(pmip_v,experiment_name,variable_name,savedirct,"mean")


# both: save and return the outputs from both "model" and "mean".
# model_list,ave,std,DATA = calculation_ensemble_change(pmip_v,experiment_name,variable_name,"both")

In [ ]:
# e.g.
model_list,ave,std = calculation_ensemble_change(pmip_v,experiment_name,variable_name,choice)

# Section 3: Plotting

This section gives some defined plotting setups that were used in Brierley et al. (2020) and Otto-Bliesner et al. (2021)

In [ ]:
### Plotting 

In [ ]:
# Temperature
#abs temp
def axplot1_tas(ax,modeldata,title):
    cmap1=plt.get_cmap('rainbow')
    bounds1 =np.arange(-2,40,2)
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='both')
    cb1label='Surface air temperature (%sC)' %chr(176)
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig


# PMIP4 anomalies
def axplot2_tas(ax,modeldata,title):
    cmap1=plt.get_cmap('bwr')
    bounds1 =[-1.5,-1,-0.8,-0.6,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.6,0.8,1,1.5]
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)   
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='both')
    cb1label='Ensemble mean, temperature change (%sC)' %chr(176)
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig



# std
def axplot3_tas(ax,modeldata,title):
    cmap1=plt.get_cmap('YlOrRd')
    bounds1 =np.arange(0,1.25,0.1)
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='max')
    cb1label='Ensemble standard deviation (%sC)' %chr(176)
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig



In [ ]:
# Precipitaion
#abs precipitation rate
def axplot1_pr(ax,modeldata,title):
    cmap1=plt.get_cmap('GnBu')
    bounds1 =[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10,12,14,16]
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='max')
    cb1label='Precipitation rate (mm/day)'
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig


# PMIP4 anomalies
def axplot2_pr(ax,modeldata,title):
    cmap1=plt.get_cmap('BrBG')
    bounds1 =[-1.2,-0.9,-0.6,-0.3,-0.25,-0.2,-0.15,-0.1,-0.05,0,0.05,0.1,0.15,0.2,0.25,0.3,0.6,0.9,1.2]
    #[-1,-0.8,-0.6,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.6,0.8,1]
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='both')
    cb1label='Ensemble mean, precipitation change (mm/day)'
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig



# std
def axplot3_pr(ax,modeldata,title):
    cmap1=plt.get_cmap('YlGn')
    bounds1 =np.arange(0,1.05,0.1)
    norm1 = colors.BoundaryNorm(boundaries=bounds1, ncolors=256)
    ax.set_title(title,fontsize=15)
    ax.coastlines()
    ax.gridlines()
    fig=ax.imshow(modeldata, transform=transform,cmap=cmap1,norm=norm1)
    cax1,kw1 = matplotlib.colorbar.make_axes([ax for ax in [ax]],location='bottom',pad=0.05,fraction=0.05)
    cb1=plt.colorbar(fig, cax=cax1, **kw1,extend='max')
    cb1label='Ensemble standard deviation (mm/day)' 
    cb1.set_label(cb1label,fontsize=15)
    cb1.ax.tick_params(labelsize=12)
    return fig



In [ ]:
# Plotting a single panel
projection = ccrs.Robinson()
transform=ccrs.PlateCarree()
plt.figure()
ax = plt.axes(projection=projection)
axplot1_tas(ax,modeldata,pmip_v)